In [1]:
from azureml.core import Workspace

In [2]:
subscription_id = "Add Sub ID #" # The ID of the Azure Subscription
resource_group = "gestaltml" # Name of a logical resource group
workspace_name = "fastai2" # The name of the workspace to look for or to create
workspace_region = 'East US' # Location of the workspace
#experiment_name = 'breastcancer'
score_script = 'score_and_track.py'
modelupload_name = 'breastcancerdetect_fastai2'

In [3]:
#Create Workspace if needed
# ws = Workspace.create(name=workspace_name,
#                subscription_id=subscription_id,
#                resource_group=resource_group,
#                create_resource_group=True,
#                location=workspace_region,
#                )

In [4]:

ws = Workspace.get(name=workspace_name, subscription_id=subscription_id, resource_group=resource_group)

Note, we have launched a browser for you to login. For old experience with device code, use "az login --use-device-code"
Performing interactive authentication. Please follow the instructions on the terminal.
You have logged in. Now let us find all the subscriptions to which you have access...


AttributeError: 'BasicTokenAuthentication' object has no attribute 'get_token'

Manually Upload & Register Model in Azure ML Workspace (or use register script below)

In [ ]:
#from azureml.core.model import Model

#Register model generated in Azure
# model = Model.register(model_path="export.pkl",
#                           model_name=modelupload_name,
#                           tags={'data': 'Babies', 'method':'transfer learning','framework':'pytorch'},
#                           description='Image classficiation of Baby Vibes',
#                           workspace=ws)

In [ ]:
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core import Environment
from azureml.core.model import InferenceConfig

myenv = CondaDependencies.create(pip_packages=['azureml-defaults', 'fastai==1.0.61','pillow==5.4.1',
                                               'torch', 'torchvision>=0.5.0'])

with open("myenv.yml","w") as f:
    f.write(myenv.serialize_to_string())
    
print(myenv.serialize_to_string())

myenv = Environment.from_conda_specification(name="myenv", file_path="myenv.yml")
inference_config = InferenceConfig(entry_script="score_and_track.py",
                                   environment=myenv)

In [ ]:
from azureml.core.webservice import Webservice
from azureml.core.webservice import AciWebservice
from azureml.exceptions import WebserviceException


service_name = 'breastcancerdetect_fastai'

# Remove any existing service under the same name.
try:
   Webservice(ws, service_name).delete()
except WebserviceException:
   pass
    
aciconfig = AciWebservice.deploy_configuration(cpu_cores=2, 
                                               memory_gb=4, 
                                               tags={'data': 'breastcancer',  'method':'transfer learning', 'framework':'pytorch'},
                                               description='')

In [ ]:
service = Model.deploy(workspace=ws, 
                           name=service_name, 
                           models=[model], 
                           inference_config=inference_config, 
                           deployment_config=aciconfig)
service.wait_for_deployment(True)
print(service.state)

In [ ]:
service.get_logs()

In [ ]:
print(service.scoring_uri)